---
# Homework 2 - eCommerce analytics
---

# Notes before starting

We dealt with a huge amount of data from the files `2019-Oct.csv` and `2019-Nov.csv`. These files have a total size equal to 13,4GB. For all the questions, we reduced the amount of RAM required by loading only the needed columns and, since some columns data allows it, we forced some data casting to data types smaller in size. Specifically, for example, we loaded the `user_id` values as `numpy.int32` instead of default `numpy.int64`, since we observed that `user_id` values are of 10<sup>8</sup> order.

Moreover, we tried to handle these data applying two different strategies. The first one was execute the fucntions in a AWS EC2 instance configured with the proper quantity of RAM. According to the second one, we implemented some functions chunking the dataset, computing by chunk and then combining the results. We are aware that this implementation adds a lot of overhead, but this way we managed to run these functions on a laptop with only 8GB of RAM. These functions are included in the file `./scripts/our_functions_low_memory` that we chose to use in this notebook. All the questions marked with a **\*** were solved with such functions. For these questions, we also provide a file with a more computationally efficent implementation (see `./scripts/our_functions.py`).

# Setup

As previously mentioned, in order to keep the notebook tidy, for each question, we put the functions in the file `./scripts/our_functions_low_memory.py`.

In [ ]:
import scripts.our_functions_low_memory as f

Specifically, the libraries we used are the following:

```python
# for data handling and analysis
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 
import datetime

# utilies for computational performance evaluation and debugging
from tqdm import tqdm
from threading import Thread
from queue import Queue
import os, glob
```

To have an insight into the data handling, we show the routine function to load the needed data for each question: 

```python
def get_needed_iterator(month_files:list, columns:list, chunksize:int=10**6, parse_dates=False):
    global DATA_PATH, dtype
    iterators = []
    date_parser = pd.to_datetime if parse_dates else None
    for month in month_files:
        iterators.append(pd.read_csv(os.path.join(DATA_PATH, month), usecols=columns, 
                                     parse_dates=parse_dates, date_parser=date_parser, 
                                     chunksize=chunksize, dtype=dtype))
    return iterators
```

According to the pandas API, we split the dataset into chunks of 10<sup>6</sup> rows and we obtain an iterator. Using this iterator properly, we get all the data required for each question.

---
# [RQ1]*  Which is the rate of complete funnels?

According to 